In [ ]:
import random
# The fitness function evaluates the quality of the solution.
# The higher the numeric value, the better the solution.
# Given a solution encoding of genes a, b, c, d, e, f, g, h, output a numeric value
def fitness(solution):
    (a, b, c, d, e, f, g, h) = map(int, solution)
    return (a + b) - (c + d) + (e + f) - (g + h)
# Given two solutions, parent1 and parent2, return two new solutions
# that are the result of crossing over the genes of the parents.
# The crossover point is in the middle of the solution.
def one_point_crossover(parent1, parent2):
    return parent1[:4] + parent2[4:], parent2[:4] + parent1[4:]
# Given two solutions, parent1 and parent2, return two new solutions
# that are the result of crossing over the genes of the parents.
# The crossover points are at the gene b and gene f.
def two_point_crossover(parent1, parent2):
    return parent1[:2] + parent2[2:6] + parent1[6:], parent2[:2] + parent1[2:6] + parent2[6:]
# Given two solutions, parent1 and parent2, return two new solutions
# that are the result of crossing over the genes of the parents.
# Some strategy for each gene to determine a swap
# unlike one_point_crossover and two_point_crossover that swap collections of genes
def uniform_crossover(parent1, parent2):
    child1 = ""
    child2 = ""
    for i in range(len(parent1)):
        if i % 2 == 0:
            child1 += parent1[i]
            child2 += parent2[i]
        else:
            child1 += parent2[i]
            child2 += parent1[i]
    return child1, child2
# Given two solutions, parent1 and parent2, return two new solutions
# that are the result of crossing over the genes of the parents.
# Randomly choose to swap genes between the parents
def random_crossover(parent1, parent2):
    child1 = ""
    child2 = ""
    for gene1, gene2 in zip(parent1.split(), parent2.split()):
        if random.random() < 0.5:
            child1 += gene1
            child2 += gene2
        else:
            child2 += gene1
            child1 += gene2
    return child1, child2
# choose one of the crossover strategies
def choose_crossover_strategy():
    crossover_strategies = [
        one_point_crossover,
        two_point_crossover,
        uniform_crossover,
        random_crossover
    ]
    # favor some strategies over others
    weights = [
        0.25,
        0.25,
        0.1,
        0.7
    ]
    return random.choices(crossover_strategies, weights=weights, k=1)[0]
# Given a population of solutions, select a solution that is relatively fit
# compared to the other solutions in the population.
# The probability of selecting a solution is proportional to its fitness score.
def select_relatively_fit(population):
    def sortByFitness(solution_fitness_pair):
        return solution_fitness_pair[1]
    solution_fitness_pairs = [(solution, fitness(solution)) for solution in population]
    sorted_solution_fitness_pairs = sorted(solution_fitness_pairs, key=sortByFitness)
    top_20_percent_index = int(len(sorted_solution_fitness_pairs) * 0.8)
    top_20_percent_solution_fitness_pairs = sorted_solution_fitness_pairs[top_20_percent_index:]
    top_20_percent_population = [solution for solution, _ in top_20_percent_solution_fitness_pairs]
    return random.choice(top_20_percent_population)
# Given a population of solutions, select a solution randomly
def select_randomly(population):
    return population[random.randint(0, len(population) - 1)]
# Given a population of solutions, select a solution based on a criteria function
# Generalizes selecting the fittest or least fit solution from a population
def select_solution_by_criteria(population, criteria):
    selected = population[0]
    for solution in population:
        if criteria(solution, selected):
            selected = solution
    return selected
# Given a population of solutions, select the solution with the highest fitness score
def select_fittest(population):
    return select_solution_by_criteria(
        population,
        lambda solution, selected: fitness(solution) > fitness(selected)
    )
# Given a population of solutions, select the solution with the lowest fitness score
def select_least_fit(population):
     return select_solution_by_criteria(
        population,
        lambda solution, selected: fitness(solution) < fitness(selected)
    )
# Given a population of solutions, select a solution to be the parent for the next generation of solutions
# Favor solutions that are relatively fit compared to the other solutions in the population
def select_parent(population):
    parent = select_relatively_fit(population)
    if (parent == None):
        return select_randomly(population)
    return parent
# Given a parent population and an offspring population, return a new population
# Elitism selection is a strategy that preserves some number of most fit solutions from the parent population
def elitism(parent_population, offspring_population):
    most_fit_parent = select_fittest(parent_population)
    least_fit_child = select_least_fit(offspring_population)
    offspring_population.remove(least_fit_child)
    offspring_population.append(most_fit_parent)
    return offspring_population

# Given a population of solutions, return the mean fitness score of the population
# The mean fitness can be used to determine if the population is improving over time
def mean_fitness(population):
    return sum([fitness(solution) for solution in population]) / len(population)

f_1 = "65413532"
f_2 = "87126601"
f_3 = "23921285"
f_4 = "41852094"
initial_population = [f_1, f_2, f_3, f_4]
number_of_generations = 1000000

def run(population=initial_population, number_of_generations=number_of_generations):
    # Run the genetic algorithm for a number of generations
    for _ in range(number_of_generations):
        new_population = []
        # Create a new population
        for _ in range(0, len(population), 2):
            # Select two parents
            parent1 = select_parent(population)
            parent2 = select_parent(population)
            # Cross over the parents to create two children
            crossover = choose_crossover_strategy()
            child1, child2 = crossover(parent1, parent2)
            # Add the children to the new population
            new_population.append(child1)
            new_population.append(child2)
        # Replace the old population with the new population
        # preserving the most fit solution from the parent population
        population = elitism(population, new_population)
    # Return the best solution found
    return select_fittest(population)

approximately_best_solution = run()
print(one_point_crossover("99009900", "00110011"))
print(two_point_crossover("99009900", "00110011"))
print(uniform_crossover("99009900", "00110011"))
optimal_solution = "99009900"
print(f"Optimal solution, {optimal_solution}, has a fitness score of {fitness(optimal_solution)}")
print(f"fitness score of {f_1}: {fitness(f_1)}")
print(f"fitness score of {f_2}: {fitness(f_2)}")
print(f"fitness score of {f_3}: {fitness(f_3)}")
print(f"fitness score of {f_4}: {fitness(f_4)}")
child_1, child_2 = one_point_crossover(f_2, f_1)
child_3, child_4 = two_point_crossover(f_1, f_3)
child_5, child_6 = uniform_crossover(f_2, f_3)
print("One point cross over on first and second fittest: " + child_1 + " " + child_2)
print("Two point cross over on second and third fittest: " + child_3 + " " + child_4)
print("Uniform cross over on first and third fittest: " + child_5 + " " + child_6)
print(f"fitness of child 1: {fitness(child_1)}")
print(f"fitness of child 2: {fitness(child_2)}")
print(f"fitness of child 3: {fitness(child_3)}")
print(f"fitness of child 4: {fitness(child_4)}")
print(f"fitness of child 5: {fitness(child_5)}")
print(f"fitness of child 6: {fitness(child_6)}")
print(f"Mean fitness of original population: {mean_fitness([f_1, f_2, f_3, f_4])}")
print(f"Mean fitness of next generation: {mean_fitness([child_1, child_2, child_3, child_4, child_5, child_6])}")
print(f"Best solution found: {approximately_best_solution} with a fitness score of {fitness(approximately_best_solution)}")


('99000011', '00119900')
('99110000', '00009911')
('90019001', '09100910')
Optimal solution, 99009900, has a fitness score of 36
fitness score of 65413532: 9
fitness score of 87126601: 23
fitness score of 23921285: -16
fitness score of 41852094: -19
One point cross over on first and second fittest: 87123532 65416601
Two point cross over on second and third fittest: 65921232 23413585
Uniform cross over on first and third fittest: 83126205 27921681
fitness of child 1: 15
fitness of child 2: 17
fitness of child 3: -2
fitness of child 4: -5
fitness of child 5: 11
fitness of child 6: -4
Mean fitness of original population: -0.75
Mean fitness of next generation: 5.333333333333333
Best solution found: 87126601 with a fitness score of 23
